# このノートブックはコマンドのコピペ用です
所々レクチャー時に利用します

# コマンド保存
docker exec -it pyspark_big /bin/bash

# pyspark-topicの作成コマンド

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka_big:9092 
```

# コマンド保存
今回は、コンテナ名「metabase_postgres_big」のmetabaseデータベースからordersテーブルのデータをembulkを使って取得してみましょう。

df = spark.read.parquet("/tmp/share_file/datalake/")
df.show()  

# コマンド保存
pyspark --packages org.apache.spark:spark-streaming_2.13:3.2.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4,org.apache.spark:spark-avro_2.12:3.2.4

spark-submit  --packages org.apache.spark:spark-streaming_2.13:3.2.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4,org.apache.spark:spark-avro_2.12:3.2.4  streaming.py 

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka_big:9092") \
  .option("subscribe", "pyspark-topic") \
  .load()

# メモリシンク
memory_sink = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("web_actions") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/web_memory2/") \
  .start()

spark.sql("select key,name,action,sendtime from web_actions LATERAL VIEW json_tuple(value,'name','action','sendtime') user as name, action,sendtime ")